### installations, clones and imports

In [1]:
%%capture

!pip install -q transformers datasets peft bitsandbytes wandb trl

In [ ]:
!git clone https://github.com/onyekaokonji/LLaVA-Med.git LLaVA_Med

Cloning into 'LLaVA_Med'...
remote: Enumerating objects: 352, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 352 (delta 6), reused 32 (delta 0), pack-reused 306 (from 1)
Receiving objects: 100% (352/352), 77.05 MiB | 16.89 MiB/s, done.
Resolving deltas: 100% (78/78), done.


In [ ]:
import os
os.chdir("/content/LLaVA_Med")

os.getcwd()

'/content/LLaVA_Med'

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch.utils.data.dataset import Dataset
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset
from PIL import Image
from io import BytesIO
import requests
import json
import uuid

from transformers import Trainer, TrainingArguments
from peft import LoraConfig, LoraModel, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

from llava.constants import DEFAULT_IMAGE_TOKEN, IMAGE_TOKEN_INDEX
from llava.conversation import Conversation
from llava.mm_utils import tokenizer_image_token, process_images
from llava.model.builder import load_pretrained_model
from llava.conversation import conv_templates

from huggingface_hub import notebook_login
notebook_login()

### data preparation and loading

In [ ]:
def process_and_save(dataset, output_folder, subset_name):
  subset_folder = os.path.join(output_folder, subset_name)
  image_subfolder = os.path.join(subset_folder, "images")

  if not os.path.exists(image_subfolder):
    os.makedirs(image_subfolder, exist_ok=True)

  if not os.path.exists(subset_folder):
    os.makedirs(subset_folder)

  json_data_list = []

  for item in dataset:
    if isinstance(item["image"], str):
      response = requests.get(item["image"])
      image = Image.open(BytesIO(response.content))
    else:
      image = item["image"]

    unique_id = str(uuid.uuid4())

    image_path = os.path.join(image_subfolder, f"{unique_id}.jpg")
    image.save(image_path)

    answers = item["answer"]
    formatted_answers = "".join(answers)

    json_data = {
        "id": unique_id,
        "image": f"{unique_id}.jpg",
        "conversations": [
            {
                "from": "human",
                "value": item["question"]
            },
            {
                "from": "gpt",
                "value": formatted_answers
            }
        ]
    }

    json_data_list.append(json_data)

  json_output_path = os.path.join(output_folder, subset_name, "dataset.json")
  with open(json_output_path, "w") as json_file:
    json.dump(json_data_list, json_file, indent=4)


def save_dataset(dataset_name, output_folder, subset_name):
  dataset = load_dataset(dataset_name)

  process_and_save(dataset[subset_name], output_folder, subset_name)

In [ ]:
output_folder = "/content/drive/MyDrive/dataset"
save_dataset("flaviagiammarino/vqa-rad", output_folder, 'train')
save_dataset("flaviagiammarino/vqa-rad", output_folder, 'test')

Generating train split:   0%|          | 0/1793 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/451 [00:00<?, ? examples/s]

In [ ]:
class VQARAD(Dataset):
  def __init__(self, split):
    super(VQARAD, self).__init__()
    self.split = split
    self.image_folder = f'/content/drive/MyDrive/dataset/{self.split}/images'
    self.paths = {
        'train': '/content/drive/MyDrive/dataset/train/dataset.json',
        'test': '/content/drive/MyDrive/dataset/test/dataset.json'
    }

    with open(self.paths[self.split], 'r') as f:
      self.dataset = json.load(f)

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    item = self.dataset[idx]
    id = item['id']
    question = item['conversations'][0]['value']
    answer = item['conversations'][1]['value']
    image_path = item['image']
    image = Image.open(os.path.join(self.image_folder, image_path)).convert('RGB')

    return id, question, answer, image

In [ ]:
class DataCollator:
  def __init__(self, tokenizer, split, conversation_template, pad_token_id, image_processor, model_config):
    self.tokenizer = tokenizer
    self.split = split
    self.conversation_template = conversation_template
    self.pad_token_id = pad_token_id
    self.image_processor = image_processor
    self.model_config = model_config

  def __call__(self, rows):
    if self.split == "train":
      return self._collate_train(rows)
    elif self.split == "test":
      return self._collate_test(rows)
    else:
      return ValueError(f"Invalid split: {self.split}")

  def _collate_train(self, rows):
    train_input_ids_list=[]
    train_labels_list=[]
    train_images=[]

    for row in rows:
      id, question, answer, image = row
      train_images.append(image)

      question = question.replace(DEFAULT_IMAGE_TOKEN, '').strip()
      question = DEFAULT_IMAGE_TOKEN + '\n' + question


      conv = self.conversation_template.copy()
      conv.append_message(conv.roles[0], question)
      conv.append_message(conv.roles[1], None)
      prefix = conv.get_prompt()

      conv = self.conversation_template.copy()
      conv.append_message(conv.roles[0], question)
      conv.append_message(conv.roles[1], answer)
      full = conv.get_prompt()

      prefix = tokenizer_image_token(prefix, self.tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
      full = tokenizer_image_token(full, self.tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")

      prefix_length = prefix.size(0)
      full_length = full.size(0)

      train_input_ids = full
      train_labels = full.clone()
      train_labels[:prefix_length] = -100

      train_input_ids_list.append(train_input_ids)
      train_labels_list.append(train_labels)

    pad_value = -114514
    train_input_ids = pad_sequence(train_input_ids_list, batch_first=True, padding_value=pad_value)
    train_labels = pad_sequence(train_labels_list, batch_first=True, padding_value=pad_value)
    train_attention_mask = (train_input_ids != pad_value).long()

    train_input_ids[train_input_ids == pad_value] = self.pad_token_id
    train_labels[train_labels == pad_value] = self.pad_token_id

    train_images = process_images(train_images, self.image_processor, self.model_config).to(torch.bfloat16)

    return {
        "input_ids": train_input_ids,
        "labels": train_labels,
        "attention_mask": train_attention_mask,
        "images": train_images
    }

  def _collate_test(self, rows):
    validation_input_ids_list=[]
    validation_labels_list=[]
    validation_images=[]

    for row in rows:
      id, question, answer, image = row
      validation_images.append(image)

      question = question.replace(DEFAULT_IMAGE_TOKEN, '').strip()
      question = DEFAULT_IMAGE_TOKEN + '\n' + question


      conv = self.conversation_template.copy()
      conv.append_message(conv.roles[0], question)
      conv.append_message(conv.roles[1], None)
      prefix = conv.get_prompt()

      conv = self.conversation_template.copy()
      conv.append_message(conv.roles[0], question)
      conv.append_message(conv.roles[1], answer)
      full = conv.get_prompt()

      prefix = tokenizer_image_token(prefix, self.tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
      full = tokenizer_image_token(full, self.tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")

      prefix_length = prefix.size(0)
      full_length = full.size(0)

      validation_input_ids = full
      validation_labels = full.clone()
      validation_labels[:prefix_length] = -100

      validation_input_ids_list.append(validation_input_ids)
      validation_labels_list.append(validation_labels)

    pad_value = -114514
    validation_input_ids = pad_sequence(validation_input_ids_list, batch_first=True, padding_value=pad_value)
    validation_labels = pad_sequence(validation_labels_list, batch_first=True, padding_value=pad_value)
    validation_attention_mask = (validation_input_ids != pad_value).long()

    validation_input_ids[validation_input_ids == pad_value] = self.pad_token_id
    validation_labels[validation_labels == pad_value] = self.pad_token_id

    validation_images = process_images(validation_images, self.image_processor, self.model_config).to(torch.bfloat16)

    return {
        "input_ids": validation_input_ids,
        "labels": validation_labels,
        "attention_mask": validation_attention_mask,
        "images": validation_images
    }

In [ ]:
tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path="microsoft/llava-med-v1.5-mistral-7b",
    model_base="llava-med-v1.5-mistral-7b",
    model_name="llava-med-v1.5-mistral-7b"
    )

vqa_rad_dataset_train = VQARAD(split="train")
vqa_rad_dataset_test = VQARAD(split="test")

conv = conv_templates["mistral_instruct"]

collate_fn = DataCollator(
    tokenizer=tokenizer,
    split="train",
    conversation_template=conv,
    pad_token_id=tokenizer.pad_token_id,
    image_processor=image_processor,
    model_config=model.config
)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at microsoft/llava-med-v1.5-mistral-7b were not used when initializing LlavaMistralForCausalLM: ['model.vision_tower.vision_tower.vision_model.embeddings.class_embedding', 'model.vision_tower.vision_tower.vision_model.embeddings.patch_embedding.weight', 'model.vision_tower.vision_tower.vision_model.embeddings.position_embedding.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm2.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.la

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [ ]:
model

LlavaMistralForCausalLM(
  (model): LlavaMistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): Mistr

In [ ]:
model.train()

model.gradient_checkpointing_enable()

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    target_modules=["k_proj", "q_proj", "v_proj", "out_proj"],
    lora_alpha=16
)

peft_model = get_peft_model(model, lora_config, "default")
peft_model.print_trainable_parameters()

trainable params: 6,291,456 || all params: 7,572,510,720 || trainable%: 0.0831


In [ ]:
training_args = TrainingArguments(
    output_dir="trained_llava-med",
    report_to="wandb",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    logging_steps=5,
    learning_rate=2e-5,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    warmup_ratio=0.03,
    weight_decay=0.01,
    remove_unused_columns=True,
    gradient_checkpointing=True,
    fp16=False,
    bf16=True,
    optim='paged_adamw_8bit')

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=vqa_rad_dataset_train,
    eval_dataset=vqa_rad_dataset_test,
    data_collator=collate_fn
)

peft_model.config.use_cache = False
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 37


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.070400,No log
2,0.591000,No log
3,0.442700,No log
4,0.327700,No log
5,0.243500,No log


TrainOutput(global_step=8965, training_loss=0.5350589716028914, metrics={'train_runtime': 4075.6216, 'train_samples_per_second': 2.2, 'train_steps_per_second': 2.2, 'total_flos': 1.058735921163264e+16, 'train_loss': 0.5350589716028914, 'epoch': 5.0})

In [ ]:
peft_model_id = "onyekaokonji/llava-med-lora"

model.push_to_hub(peft_model_id)
trainer.push_to_hub(peft_model_id)

model-00006-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

model-00003-of-00007.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

model-00004-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

model-00002-of-00007.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/25.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/onyekaokonji/trained_llava-med/commit/5ea920952dfcf2becb4ee3800f2f9bd5ddc9dfdf', commit_message='onyekaokonji/llava-med-lora', commit_description='', oid='5ea920952dfcf2becb4ee3800f2f9bd5ddc9dfdf', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# trainer.model.save_pretrained("/content/drive/MyDrive/trained_modelss")